<a href="https://colab.research.google.com/github/gvanathip/Thai-Travel-Open-AI-Chatbot-with-GPT2/blob/main/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
n = 15
#number of previous sentences

In [ ]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np

import string 

In [ ]:
pip install pythainlp[full]

In [ ]:
paths = ['/content/drive/MyDrive/IS-DS/BarbieDreamhouseAdventuresGoTeamRobertsTH',
         '/content/drive/MyDrive/IS-DS/BarbieDreamhouseAdventuresTH',
         '/content/drive/MyDrive/IS-DS/DiaryOfTootsiesTH',
         '/content/drive/MyDrive/IS-DS/HormonesTH',
         '/content/drive/MyDrive/IS-DS/PenTor2019TH',
         '/content/drive/MyDrive/IS-DS/TalkingTomAndFriendsTH',
         '/content/drive/MyDrive/IS-DS/TheBossBabyTH',
         '/content/drive/MyDrive/IS-DS/WakeUpLadiesTheSeriesTH',
         '/content/drive/MyDrive/IS-DS/YouTube']

In [ ]:
detects = ['คำบรรยายโดย'
  ,'ติดตามตอนต่อไป'
  ,'ความเดิมตอนที่แล้ว'
  ,'♪'
  ,'เสียง'
  ,'เอฟเฟ็ค'
  ,'ดนตรี'
  ,'เอฟเฟกต์']

In [ ]:
from pythainlp.wangchanberta import ThaiNameTagger

In [ ]:
ner = ThaiNameTagger()

In [ ]:
from csv import DictWriter
field_names = ['fname','len']

In [ ]:
def extract(path_,fnames_):
    df = pd.DataFrame()

    # for fname in fnames_:

    #     print(fname)

    dialog = []

    lines = []
    with open(path_+'/'+fnames_) as f:
        lines = f.readlines()

    count = 0
    for line in lines:
            
        if ('<c.thai><c.bg_transparent>' in line) & all(detect not in line for detect in detects):
              count += 1
              line = re.sub(r'<.*?>','',line)
              line = line.replace('<c.thai><c.bg_transparent>','')
              line = line.replace('</c.bg_transparent></c.thai>','')
              line = line.replace('\n','')
              line = line.replace('<c.thai>','')
              line = line.replace('</c.thai>','')
              line = line.replace('<c.bg_transparent>&lrm;','')
              line = line.replace('&lrm;','')
              line = line.replace('</c.bg_transparent>','')
              line = line.replace('<c.bg_transparent>','')
              line = line.replace('- ','')

              line = line.replace('กู','เรา')
              line = line.replace('มีง','name')

              for c in string.punctuation:
                line = re.sub(r'\{}'.format(c),'',line)



              line = line.replace('à¹','')
              line = line.replace('à','')
              line = line.replace('Ī','')
              line = line.replace('ī','')
              line = line.replace('±','')
              line = line.replace('µ','')
              line = line.replace('´','')
              line = line.replace('Ļ','')
              line = line.replace('°','')
              line = line.replace('§','')
              line = line.replace('¢','')

              line = line.replace('เจ๊กล้วย','name')
              line = line.replace('อีคิม','name')
              line = line.replace('คิม','name')
              line = line.replace('อีกัส','name')
              line = line.replace('กัส','name')
              line = line.replace('อีกอล์ฟ','name')
              line = line.replace('กอล์ฟ','name')
              line = line.replace('อีช้าง','name')
              line = line.replace('พี่หมอน','name')
              line = line.replace('น้องวาว','name')
              line = line.replace('พี่เบียร์','name')
              line = line.replace('พี่กอล์ฟ','name')
              line = line.replace('พี่ยม','name')
              line = line.replace('ยม','name')
              line = line.replace('แองเจล่า','name')
              line = line.replace('น้องพอใจ','name')
              line = line.replace('จินเจอร์','name')
              line = line.replace('น้องขั้นเทพ','name')
              line = line.replace('ออย','name')
              line = line.replace('หมอน','name')
              line = line.replace('อีแหนด','name')
              line = line.replace('เอ๋ย','name')
              line = line.replace('บอสเบบี้','name')
              line = line.replace('เบบี้บอส','name')
              line = line.replace('เจ๊มิ้นท์','name')
              line = line.replace('เจ๊','name')
              line = line.replace('พี่ต่อ','name')
              line = line.replace('พี่อู๊ด','name')
              line = line.replace('พี่ติด','name')
              line = line.replace('อีกล้วย','name')
              line = line.replace('อีเบน','name')
              line = line.replace('หมอก','name')
              line = line.replace('ดาว','name')
              line = line.replace('หนิง','name')
              line = line.replace('ไอ้เรือง','name')
              line = line.replace('เต้ย','name')

              if len(line)>0:
                if (line[0] != '(') & (line[-1] != ')') & all(detect not in line for detect in detects) & (line[0] != '[') & (line[-1] != ']') :
                  temp = ner.get_ner(line, tag=True)
                  if '<PERSON>' in temp:
                    temp = re.sub(r'<PERSON>(.*?)</PERSON>', 'name', temp)
                  else:
                    temp = line

                  line = re.sub(r'<(.*?)>', '', temp)
                  line = re.sub(r"[-()\"#/@;:<>{}=~|.?,]", "", line)
                  line = line.replace('�','')

                  print(line)
                  dialog.append(line)
                #print(f'line {count}: {line}')    

    #print(dialog)

    contexted = []
    for i in range(n, len(dialog)):
      row = []
      prev = i - 1 - n # we additionally subtract 1, so row will contain current response and 7 previous responses  
      for j in range(i, prev, -1):
        row.append(dialog[j])
      contexted.append(row)
    columns = ['response', 'context'] 
    columns = columns + ['context/'+str(i) for i in range(n-1)]
    df = df.append(pd.DataFrame.from_records(contexted, columns=columns))

    # print(path_+'/'+fnames_)
    # print(len(dialog))
    # print(df.head())


    # Dictionary
    dict_={'fname':path_+'/'+fnames_,'len':len(dialog)}
      
    # Open your CSV file in append mode
    # Create a file object for this file
    with open('/content/drive/MyDrive/IS-DS/len_info.csv', 'a') as f_object:
          
        # Pass the file object and a list 
        # of column names to DictWriter()
        # You will get a object of DictWriter
        dictwriter_object = DictWriter(f_object, fieldnames=field_names)
      
        #Pass the dictionary as an argument to the Writerow()
        dictwriter_object.writerow(dict_)
      
        #Close the file object
        f_object.close()


    return df

In [ ]:
prep_df = pd.DataFrame()

for path in paths:
  #print(path)
  fnames = os.listdir(path)
  fnames.sort()
  #print(fnames)

  for fname in fnames[1:]:
    prep_df = pd.concat([prep_df,extract(path_=path,fnames_=fname)])

prep_df.to_csv('/content/drive/MyDrive/IS-DS/prep_df'+str(n)+'.csv',index=False)